<hr/>

# Introduction to Data Science
**Tamás Budavári** - budavari@jhu.edu <br/>

- Eigensystem of covariance matrix
- Fitting lines and planes

<hr/>

<h1><font color="darkblue">Recap from last time</font></h1>

### Eigendecomposition 

> $ C = E\,\Lambda\,E^T$ and $E^T C\,E = \Lambda$ since $E^T E=I$
><br/><br/>
> or
><br/><br/>
>$\displaystyle C = \sum_{k=1}^N\ \lambda_k\left(\boldsymbol{e}_k\,\boldsymbol{e}_k^T\right) 
= \sum_{k=1}^N\ \lambda_k P_k $

### Scree Plot

- The eigenvalue spectrum

>$ \big\{ \lambda_1, \lambda_2, \dots, \lambda_N \big\}$

- How many important directions?

> Keep $K =\,?$ principal components

- Explained variance of components 

> Cf. $\mathbb{Var}[X\pm{}Y] = \mathbb{Var}[X]+\mathbb{Var}[Y]$ if uncorrelated

#### Generate data

In [1]:
%pylab inline
pylab.rcParams['figure.figsize'] = (4,4)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.stats import norm as gaussian
import pandas as pd

In [ ]:
# generate 10-D vectors: scale, rotate
np.random.seed(1)
Z = gaussian.rvs(0,1,(10,1000))
if True: # scale them here
    for i in range(Z[:,0].size): 
        Z[i,:] *= sqrt(i)
    Z[:4,:] *= 1e-7
    
# quick-n-dirty random rotation
M = random.randn(Z[:,0].size,Z[:,0].size)
Q,_ = np.linalg.qr(M) # QR decomposition
Y = Q @ Z # random rotation
print (Y.shape)
np.savetxt("temp.csv", Y.T, delimiter=",")

In [ ]:
# remove all previous variables from memory
del Y, M, Q, Z

In [ ]:
try:
    print ("Shape of Y" % Y.shape)
except NameError as e:
    print ("Error message: %s" % e)    

#### Analyze data

Now your data file is available here: [temp.csv](temp.csv)

In [ ]:
# pandas dataframe - table data structure
df = pd.read_csv('temp.csv',header=None)
df[:3] # slice like arrays

In [ ]:
df.sample(10)

In [ ]:
# re-load the Y matrix and convert dataframe to matrix
Y = pd.read_csv('temp.csv',header=None).values.T
Y.shape

In [ ]:
subplot(111,aspect='equal')
plot(Y[0,:], Y[6,:], '.', alpha=0.2);

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=Y.shape[0], whiten=False)
B = pca.fit_transform(Y.T).T
E, L = pca.components_.T, pca.explained_variance_

subplot(1,1,1, aspect='equal')
plot(B[0,:], B[5,:], '.', alpha=0.2);

In [ ]:
subplot(2,1,1); plot(L,'o-'); ylabel('Eigenvalues');
subplot(2,1,2); cl=np.cumsum(L); ylabel('Total Variance');
plot(cl/cl[-1],'o-r'); ylim(0,None);

In [ ]:
# plot orig and new covariance matrices (estimate w/o norm)
ax=subplot(1,2,1); ax.matshow(Y @ Y.T); #Y.dot(Y.T) or np.dot(Y,Y.T)
ax=subplot(1,2,2); ax.matshow(B @ B.T);

In [ ]:
A = B[:6,:]
# plot new covariance matrices with and without truncation
ax=subplot(121); ax.matshow(B @ B.T);
ax=subplot(122); ax.matshow(A @ A.T);

### Inverse of the Covariance Matrix

- Appears in the multivariate normal distribution!

>$\displaystyle{\cal{}N}(x;\mu,C) = \frac{1}{\sqrt{\det(2\pi{}C)}}\ \exp\left[-\frac{1}{2}(x\!-\!\mu)^T\,C^{-1} (x\!-\!\mu)\right]$

- Inverse of the diagonal eigenvalue matrix

>$\displaystyle \Lambda^{-1} =  \left( \begin{array}{ccc}
\frac{1}{\lambda_1} &  & \cdots & 0\\
 & \frac{1}{\lambda_2} &   & \vdots\\
\vdots &  & \ddots &  \\
0 & \cdots &  & \frac{1}{\lambda_N} \\
\end{array} \right)$

- Inverse of the covariance matrix

>$\displaystyle C^{-1} = E\ \Lambda^{-1} E^T$

- Also see pseudoinverse with small eigenvalues 

### Fitting Lines

- What if $x$ and $y$ are both noisy? 

> For example, $\big\{(x_i,y_i)\big\}$ measurements have the same uncertainties. 
> The relevant residuals are perpendicular to the line.
> Minimizing RMS of residuals is related to maximizing the sample variance along line!

- Sounds like the PCA problem?


### Fitting Planes

- Similarly, fitting a $K$-dimensional hyperplane in $N$ dimensions, i.e., looking for the $a$ normal vector to minimize residuals on set of centered $\{x_i\}$ vectors

> Minimizing sum of square lengths of the residual vectors
><br/><br/>
>$\displaystyle \qquad \min_a \sum_i r_i^2(a) \ \ \ \ \ $  where $\ \ \ r_i(a) = x_i - \left(a\,a^T\right)x_i$, 
><br/><br/>
> yields 
><br/><br/>
>$\displaystyle \qquad \min_a \ \left[\textrm{const} - \sum_i a^T\!\!\left(x_i x_i^T\right) a\right]$
><br/>
or
><br/>
>$\displaystyle \qquad \max_a \ a^T\!\left(\sum_i x_i x_i^T\right)\, a $ 
><br/><br/>
> cf. sample variance along $a$, if data already centered

- Essentially same as the PCA problem!

In [ ]:
# generate 2D (column) vectors
np.random.seed(seed=42);
N = gaussian.rvs(0,1,(2,50)); 
N[0,:] *= 2 
f = +pi/6   # rotate by 30 deg
R = array([[cos(f), -sin(f)],
           [sin(f),  cos(f)]]) 
X = R @ N

figure(figsize=(5,5)); subplot(111,aspect='equal');
plot(X[0,:],X[1,:],'x',alpha=0.6);

In [ ]:
# project on 1st pricipal component
X -= X.mean(axis=1, keepdims=True) 
E,_,_ = linalg.svd(X) # only the eigenvectors
F = E[:,:1]           # truncated basis: only PC1
P = F @ F.T @ X       # projection
R = X - P             # residuals

figure(figsize=(5,5)); subplot(111,aspect='equal');
plot(X[0,:],X[1,:],'xb',alpha=0.6);
plot(P[0,:],P[1,:],'or',alpha=0.4);
quiver(P[0,:],P[1,:],R[0,:],R[1,:], alpha=0.2,
    angles='xy',scale_units='xy',scale=1);

### More on Fitting Later

- Next: Bayesian inference